In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Ścieżka do głównego folderu z danymi
base_dir = r"C:\Users\matik\Documents\Data Sets\Faulty_solar_panel"

# Przygotowanie generatorów danych (treningowych i walidacyjnych)
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    base_dir, 
    target_size=(224, 224),  # Rozmiar wejściowy dla ResNet50
    batch_size=32,
    class_mode='categorical',  # Klasyfikacja wieloklasowa
    subset='training'  # Używamy 80% danych do treningu
)

valid_gen = datagen.flow_from_directory(
    base_dir, 
    target_size=(224, 224),  # Rozmiar wejściowy dla ResNet50
    batch_size=32,
    class_mode='categorical',  # Klasyfikacja wieloklasowa
    subset='validation'  # Używamy 20% danych do walidacji
)


Found 711 images belonging to 6 classes.
Found 174 images belonging to 6 classes.


In [13]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50

# Ładowanie wstępnie wytrenowanego modelu ResNet50 bez głowy (bo dodamy swoje warstwy)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Zamrażanie warstw bazowych (żeby nie trenować ich ponownie)
base_model.trainable = False

# Budowanie modelu
model = models.Sequential([
    base_model,  # Wstępnie wytrenowany model ResNet50
    layers.GlobalAveragePooling2D(),  # Agregacja cech w wektorze
    layers.Dense(512, activation='relu'),  # Warstwa gęsta
    layers.Dense(train_gen.num_classes, activation='softmax')  # Wyjście z klasyfikacją
])

# Kompilacja modelu
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Podsumowanie modelu
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d_3   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_6 (Dense)             (None, 512)               1049088   
                                                                 
 dense_7 (Dense)             (None, 6)                 3078      
                                                                 
Total params: 24,639,878
Trainable params: 1,052,166
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
# Trenowanie modelu
history = model.fit(
    train_gen,
    steps_per_epoch=train_gen.samples // train_gen.batch_size,
    epochs=25,
    validation_data=valid_gen,
    validation_steps=valid_gen.samples // valid_gen.batch_size
)


Epoch 1/25
22/22 [==============================] - 39s 2s/step - loss: 1.8053 - accuracy: 0.2445 - val_loss: 1.5641 - val_accuracy: 0.3125
Epoch 2/25
12/22 [===============>..............] - ETA: 13s - loss: 1.4831 - accuracy: 0.4401

In [11]:
# Ocena modelu
loss, accuracy = model.evaluate(valid_gen)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")


6/6 [==============================] - 7s 1s/step - loss: 1.2521 - accuracy: 0.4655
Loss: 1.252073884010315
Accuracy: 0.4655172526836395


In [ ]:
model.save('resnet50_solar_fault_detection_model.h5')